In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.cluster as cluster

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;
rc_params["axes.spines.right"] = false
rc_params["axes.spines.top"] = false

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/review_figures/unimode"

In [ ]:
experiment_filename_1 = "20220406_111526"
server_1 = 9
experimenter = "jen"

In [ ]:
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

In [ ]:
NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
global Z_all = HDF5.readmmap(NMF_file["Z_all"])
global X_all = HDF5.readmmap(NMF_file["X_all"])
global Y_all = HDF5.readmmap(NMF_file["Y_all"])
global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
global A_dFF_1 = HDF5.readmmap(NMF_file["A_dFF"]); # bs update
global A_baseline_1 = HDF5.readmmap(NMF_file["A_baseline"]); # bs update
close(NMF_file)


n_neuron = length(X_all);

# whether individual roi belongs to a certain region
region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
region_bool_file = h5open(region_bool_filename, "r")
global region_names = read(region_bool_file, "region_names")
global region_roi_bool = read(region_bool_file, "region_roi_bool")
close(region_bool_file)


# for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
whether_tel = falses(n_neuron)
for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
    if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
        whether_tel[which_neuron] = true
    end
end


all_files = readdir(joinpath(data_path(ds_save_cy_1)))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*!occursin("shuffleall", candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(info_filename, "r")
place_map_all_1_old = HDF5.readmmap(file["place_map_all"])
specificity_1 = HDF5.readmmap(file["specificity"])
valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
entropy_1 = HDF5.readmmap(file["entropy_all"])
place_cell_index_1 = HDF5.readmmap(file["place_cell_index"])
close(file)

    


img_bg_1 = h5open(ds_save_1, "behavior.h5"; raw = true) do file
    read(file, "img_bg")
end;


global img_bg_end_1 = img_bg_1[:,:,end]


global w = size(img_bg_end_1, 1)
global l = size(img_bg_end_1, 2)
    
    


In [ ]:
all_files = readdir(joinpath(data_path(ds_save_cy_1) ,"oblique_corrected"))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*!occursin("shuffleall", candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(data_path(ds_save_cy_1), "oblique_corrected",save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])

close(file)
    

In [ ]:
n_bins= 60

In [ ]:
# whether individual roi belongs to a certain region
save_file_name = "compare_map_results_original.h5"
compare_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(compare_filename, "r")
confined_place_cell_index_1 = HDF5.readmmap(file["confined_place_cell_index_1"])
confined_place_cell_index_2 = HDF5.readmmap(file["confined_place_cell_index_2"])
tel_place_cell_index  = HDF5.readmmap(file["tel_place_cell_index"])
peak_loc_map_1 = HDF5.readmmap(file["peak_loc_map_1"])
peak_loc_map_2 = HDF5.readmmap(file["peak_loc_map_2"])
precise_loc = false
if haskey(file, "peak_loc_map_1_precise")
    precise_loc = true
    peak_loc_map_1_precise_array = HDF5.readmmap(file["peak_loc_map_1_precise"])
    peak_loc_map_2_precise_array = HDF5.readmmap(file["peak_loc_map_2_precise"])
end
close(file)



plot_loc(confined_place_cell_index_1; valid_neurons = valid_roi_1, label= length(confined_place_cell_index_1))



In [ ]:
for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_1), "oblique_corrected", "for_place_calculation_chamber_geometry_$(experiment_filename_1)_n$(n_bins)_oblique_corrected.h5"))
x_bins = read(for_place_calculation_file,"x_bins")
y_bins = read(for_place_calculation_file,"y_bins")
chamber_roi_1 = read(for_place_calculation_file,"chamber_roi")
close(for_place_calculation_file)

# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))

countour_array_1 = read(orientation_correction_file,"countour_array")
center_loc_1 = read(orientation_correction_file,"center_loc")
close(orientation_correction_file)


In [ ]:
data_path(ds_save_cy_1)

In [ ]:
which_map = place_map_all_1[:,:,numpy.random.choice(place_cell_index_1)]



components_peaks, img_label_valid, valid_components = map_components_peak(which_map; threshold = 8/10, components_size_threshold = 20)
for i in 1:length(components_peaks)
    scatter(components_peaks[i][2], components_peaks[i][1], color="r")
end


imshow(which_map)


In [ ]:
components_peaks, img_label_valid, valid_components = map_components_peak(which_map; threshold = 8/10, components_size_threshold = 20)
valid_components

In [ ]:
which_neuron = numpy.random.choice(confined_place_cell_index_1, 50, replace=false)
which_map = place_map_all_1[:,:,which_neuron];

figure(figsize = (40,5*ceil(Int64, size(which_map,3)/10)), dpi=250)

for i in 1:size(which_map,3)
    subplot(ceil(Int64, size(which_map,3)/10), 10,i)
    imshow(which_map[:,:,i]') 
    
    components_peaks, img_label_valid, valid_components = map_components_peak(which_map[:,:,i]; threshold = 8/10, components_size_threshold = 20)
    for i in 1:length(components_peaks)
        scatter(components_peaks[i][1], components_peaks[i][2], color="r")
    end


    axis("off")
    cell_index = which_neuron[i]

    peak_nr = length(components_peaks)
    title("$(cell_index), $(peak_nr) peak" )
end




# Check peak consistancy

In [ ]:
for threshold = [5/10, 6/10, 7/10, 8/10]

    which_map = place_map_all_1[:,:,45869]
    
    
    
    components_peaks, img_label_valid, valid_components = map_components_peak(which_map; threshold = threshold, components_size_threshold = 16/threshold)
    println(valid_components)
end

In [ ]:
allequal(x) = all(y->y==x[1],x)

In [ ]:
peak_nr_consistency = fill(false, n_neuron)
peak_identity_consistency = fill(true, n_neuron)
peak_nr = fill(NaN32, n_neuron)

In [ ]:
i = numpy.random.choice(place_cell_index_1)

which_map = place_map_all_1[:,:,i]
nr_peak = fill(NaN32, 4)

threshold = 5/10
components_peaks, img_label_valid_gt, valid_components = map_components_peak(which_map; threshold = threshold, components_size_threshold = 16/threshold)

figure()
imshow(which_map)

figure()
imshow(img_label_valid_gt)
for (i_threshold, threshold) in enumerate([5/10, 6/10, 7/10, 8/10])
    components_peaks, img_label_valid, valid_components = map_components_peak(which_map; threshold = threshold, components_size_threshold = 16/threshold)
    nr_peak[i_threshold] = length(valid_components)
    figure()
    imshow(img_label_valid)
    println(allequal([allequal(img_label_valid_gt[img_label_valid.==component]) for component in valid_components]))

    if ! allequal([allequal(img_label_valid_gt[img_label_valid.==component]) for component in valid_components])
        peak_identity_consistency[i] = false
    end

    old_component_id = [img_label_valid_gt[img_label_valid.==component][1] for component in valid_components]

    if length(numpy.unique(old_component_id)) != length(old_component_id)
        peak_identity_consistency[i] = false
    end
    
    println(old_component_id)
    println(length(numpy.unique(old_component_id)))
    
end


println(peak_identity_consistency[i])

In [ ]:
peak_nr_consistency = fill(false, n_neuron)
peak_identity_consistency = fill(true, n_neuron)
peak_nr = fill(NaN32, n_neuron)
@showprogress for i in 1:n_neuron
    which_map = place_map_all_1[:,:,i]
    nr_peak = fill(NaN32, 4)
    
    threshold = 5/10
    components_peaks, img_label_valid_gt, valid_components = map_components_peak(which_map; threshold = threshold, components_size_threshold = 16/threshold)
    
    for (i_threshold, threshold) in enumerate([5/10, 6/10, 7/10, 8/10])
        components_peaks, img_label_valid, valid_components = map_components_peak(which_map; threshold = threshold, components_size_threshold = 16/threshold)
        nr_peak[i_threshold] = length(valid_components)

    
        if ! allequal([allequal(img_label_valid_gt[img_label_valid.==component]) for component in valid_components])
            peak_identity_consistency[i] = false
        end
    
        old_component_id = [img_label_valid_gt[img_label_valid.==component][1] for component in valid_components]
    
        if length(numpy.unique(old_component_id)) != length(old_component_id)
            peak_identity_consistency[i] = false
        end

        
    end

    peak_nr_consistency[i] = allequal(nr_peak)

    if peak_nr_consistency[i]
        peak_nr[i] = nr_peak[1]
    end
end


In [ ]:
chamber_roi_mask = Float32.(copy(chamber_roi_1))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32;


valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

In [ ]:
mask_valid = isfinite.(place_map_all_1[:,:,1])

mask_valid_index = findall(mask_valid)
mask_valid_x = [x[1] for x in mask_valid_index]
mask_valid_y = [x[2] for x in mask_valid_index];
mask_valid_x_min = minimum(mask_valid_x)
mask_valid_x_max = maximum(mask_valid_x)
mask_valid_y_min = minimum(mask_valid_y)
mask_valid_y_max = maximum(mask_valid_y);

In [ ]:
which_neuron = numpy.random.choice(intersect(findall(whether_tel), place_cell_index_1, findall(peak_identity_consistency), findall(peak_nr .== 1), findall(specificity_1.>0.15)), 10, replace=false)
which_map = place_map_all_1[:,:,which_neuron];

figure(figsize = (40,5*ceil(Int64, size(which_map,3)/10)), dpi=250)

for i in 1:size(which_map,3)
    subplot(ceil(Int64, size(which_map,3)/10), 10,i)
    imshow(which_map[:,:,i]', extent = [x_bins[1], x_bins[end], y_bins[1], y_bins[end]], origin="lower")
    xlim(xmin, xmax)
    ylim(ymax, ymin)    
    components_peaks, img_label_valid, valid_components = map_components_peak(which_map[:,:,i]; threshold = 8/10, components_size_threshold = 20)
    for i in 1:length(components_peaks)
        scatter(components_peaks[i][1], components_peaks[i][2], color="r")
    end


    axis("off")
    cell_index = which_neuron[i]
    title(cell_index, fontsize=20)

end

In [ ]:
chosen_neuron = [68685, 43979, 7895, 64041, 70610, 30033, 24065, 40953]

In [ ]:
which_map = place_map_all_1[:,:,chosen_neuron];

fig = figure(dpi=3000)
for i in 1:size(which_map,3)
    subplot(3*ceil(Int64, size(which_map,3)/10), 10, i+20*floor(Int32, (i-1)/10))
    imshow(which_map[:,:,i]', extent = [x_bins[1], x_bins[end], y_bins[1], y_bins[end]], cmap="jet", origin="lower")
    imshow(chamber_roi_mask', cmap="binary", origin="lower")
    axis("off")
    xlim(xmin, xmax)
    ylim(ymax, ymin)
end


fig.savefig(joinpath(fig_path, "place_cells_unimode.png"), bbox_inches = "tight",transparent = true)

In [ ]:
which_neuron = numpy.random.choice(intersect(findall(whether_tel), place_cell_index_1, findall(peak_identity_consistency), findall(peak_nr .== 2)), 10, replace=false)
which_map = place_map_all_1[:,:,which_neuron];

figure(figsize = (40,5*ceil(Int64, size(which_map,3)/10)), dpi=250)

for i in 1:size(which_map,3)
    subplot(ceil(Int64, size(which_map,3)/10), 10,i)
    imshow(which_map[:,:,i]', extent = [x_bins[1], x_bins[end], y_bins[1], y_bins[end]], origin="lower")
    xlim(xmin, xmax)
    ylim(ymax, ymin)    
    components_peaks, img_label_valid, valid_components = map_components_peak(which_map[:,:,i]; threshold = 8/10, components_size_threshold = 20)
    for i in 1:length(components_peaks)
        scatter(components_peaks[i][1], components_peaks[i][2], color="r")
    end


    axis("off")
    cell_index = which_neuron[i]
    title(cell_index, fontsize=20)

end

In [ ]:
chosen_neuron = [63841,41648, 27354, 64848, 35066, 806, 4108, 39729]

In [ ]:
which_map = place_map_all_1[:,:,chosen_neuron];

fig = figure(dpi=3000)
for i in 1:size(which_map,3)
    subplot(3*ceil(Int64, size(which_map,3)/10), 10, i+20*floor(Int32, (i-1)/10))
    imshow(which_map[:,:,i]', extent = [x_bins[1], x_bins[end], y_bins[1], y_bins[end]], cmap="jet", origin="lower")
    imshow(chamber_roi_mask', cmap="binary", origin="lower")
    axis("off")
    xlim(xmin, xmax)
    ylim(ymax, ymin)
end


fig.savefig(joinpath(fig_path, "place_cells_multimode.png"), bbox_inches = "tight",transparent = true)

In [ ]:
which_neuron = numpy.random.choice(intersect(findall(whether_tel), place_cell_index_1, findall(peak_identity_consistency.==0)), 20, replace=false)
which_map = place_map_all_1[:,:,which_neuron];

figure(figsize = (40,5*ceil(Int64, size(which_map,3)/10)), dpi=250)

for i in 1:size(which_map,3)
    subplot(ceil(Int64, size(which_map,3)/10), 10,i)
    imshow(which_map[:,:,i]') 
    
    components_peaks, img_label_valid, valid_components = map_components_peak(which_map[:,:,i]; threshold = 8/10, components_size_threshold = 20)
    for i in 1:length(components_peaks)
        scatter(components_peaks[i][1], components_peaks[i][2], color="r")
    end


    axis("off")
    cell_index = which_neuron[i]
    title(cell_index)

end

# Examples

In [ ]:
chamber_roi_mask = Float32.(copy(chamber_roi_1))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32;


valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

In [ ]:
mask_valid = isfinite.(place_map_all_1[:,:,1])

mask_valid_index = findall(mask_valid)
mask_valid_x = [x[1] for x in mask_valid_index]
mask_valid_y = [x[2] for x in mask_valid_index];
mask_valid_x_min = minimum(mask_valid_x)
mask_valid_x_max = maximum(mask_valid_x)
mask_valid_y_min = minimum(mask_valid_y)
mask_valid_y_max = maximum(mask_valid_y);

In [ ]:
imshow(chamber_roi_mask)

In [ ]:
# Unimode
i = 16022

which_map = place_map_all_1[:,:,i]


fig = figure()
imshow(which_map', extent = [x_bins[1], x_bins[end], y_bins[1], y_bins[end]], cmap="jet", origin="lower")
imshow(chamber_roi_mask', cmap="binary", origin="lower")
axis("off")
xlim(xmin, xmax)
ylim(ymax, ymin)
fig.savefig(joinpath(fig_path, "unimode_example.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)

for (i_threshold, threshold) in enumerate([5/10, 6/10, 7/10, 8/10])
    components_peaks, img_label_valid, valid_components = map_components_peak(which_map; threshold = threshold, components_size_threshold = 20/threshold)
    img_label_valid = float32.(img_label_valid)
    img_label_valid[isnan.(which_map)] .= NaN32;
    img_label_valid[img_label_valid.>2] .= 2
    
    fig = figure()
    imshow(img_label_valid', extent = [x_bins[1], x_bins[end], y_bins[1], y_bins[end]], origin="lower")
    imshow(chamber_roi_mask', cmap="binary", origin="lower")
    axis("off")

    xlim(xmin, xmax)
    ylim(ymax, ymin)

    fig.savefig(joinpath(fig_path, "unimode_example_threshold_$(i_threshold).png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end

In [ ]:
# multimode
i = 64848

which_map = place_map_all_1[:,:,i]


fig = figure()
imshow(which_map', extent = [x_bins[1], x_bins[end], y_bins[1], y_bins[end]], cmap="jet", origin="lower")
imshow(chamber_roi_mask', cmap="binary", origin="lower")
axis("off")
xlim(xmin, xmax)
ylim(ymax, ymin)
fig.savefig(joinpath(fig_path, "multimode_example.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)

for (i_threshold, threshold) in enumerate([5/10, 6/10, 7/10, 8/10])
    components_peaks, img_label_valid, valid_components = map_components_peak(which_map; threshold = threshold, components_size_threshold = 20/threshold)
    img_label_valid = float32.(img_label_valid)
    img_label_valid[isnan.(which_map)] .= NaN32;
    img_label_valid[img_label_valid.>2] .= 2
    
    fig = figure()
    imshow(img_label_valid', extent = [x_bins[1], x_bins[end], y_bins[1], y_bins[end]], origin="lower")
    imshow(chamber_roi_mask', cmap="binary", origin="lower")
    axis("off")

    xlim(xmin, xmax)
    ylim(ymax, ymin)

    fig.savefig(joinpath(fig_path, "multimode_example_threshold_$(i_threshold).png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end

In [ ]:
# ambiguous
i = 48899

which_map = place_map_all_1[:,:,i]


fig = figure()
imshow(which_map', extent = [x_bins[1], x_bins[end], y_bins[1], y_bins[end]], cmap="jet", origin="lower")
imshow(chamber_roi_mask', cmap="binary", origin="lower")
axis("off")
xlim(xmin, xmax)
ylim(ymax, ymin)
fig.savefig(joinpath(fig_path, "ambiguous_mode_example.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)

for (i_threshold, threshold) in enumerate([5/10, 6/10, 7/10, 8/10])
    components_peaks, img_label_valid, valid_components = map_components_peak(which_map; threshold = threshold, components_size_threshold = 20/threshold)
    img_label_valid = float32.(img_label_valid)
    img_label_valid[isnan.(which_map)] .= NaN32;
    img_label_valid[img_label_valid.>2] .= 2
    
    fig = figure()
    imshow(img_label_valid', extent = [x_bins[1], x_bins[end], y_bins[1], y_bins[end]], origin="lower")
    imshow(chamber_roi_mask', cmap="binary", origin="lower")
    axis("off")

    xlim(xmin, xmax)
    ylim(ymax, ymin)

    fig.savefig(joinpath(fig_path, "ambiguous_mode_example_threshold_$(i_threshold).png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end

# unimode vs threshold

In [ ]:
thresholds = numpy.linspace(0, 8/10, 100)
nr_peak = fill(NaN32, n_neuron, length(thresholds))
@showprogress for i in place_cell_index_1
    which_map = place_map_all_1[:,:,i]

    for (i_threshold, threshold) in enumerate(thresholds)
        components_peaks, img_label_valid_gt, valid_components = map_components_peak(which_map; threshold = threshold, components_size_threshold = 20)
        nr_peak[i,i_threshold] = length(components_peaks)
    end

end


In [ ]:
uni_peak_ratio = (sum(nr_peak.==1, dims = 1)/length(place_cell_index_1))[1,:];

In [ ]:
# figure(figsize=(1,1))
plot(thresholds, uni_peak_ratio)


# fig.savefig(os.path.join(fig_path, "underlying_map.pdf"), bbox_inches="tight",transparent = True,pad_inches = 0);

# For all fish

In [ ]:
data_all_exp = 
[
    ["20220407_152537", 4, "jen"],
    ["20220406_111526", 9, "jen"],
    ["20220407_090156", 5, "jen"],
    ["20220417_165530", 2, "jen"],
    ["20220406_153842", 9, "jen"],
    ["20220405_171444", 4, "jen"],
    ["20220416_160516", 6, "jen"]
];

In [ ]:
for data_info in data_all_exp

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
    experimenter = data_info[end]
    
    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    
    all_files = readdir(data_path(ds_save_cy_1))
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*!occursin("shuffleall", candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    println(candidate_filename[which_file])
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    specificity_1 = HDF5.readmmap(file["specificity"])
    valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
    entropy_1 = HDF5.readmmap(file["entropy_all"])
    place_cell_index_1 = HDF5.readmmap(file["place_cell_index"])
    specificity_population_z = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z = HDF5.readmmap(file["specificity_shuffle_z"])
    close(file)
    
    println(minimum(specificity_population_z[place_cell_index_1]))
    println(minimum(specificity_shuffle_z[place_cell_index_1]))

end

In [ ]:
for data_info in data_all_exp

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
    experimenter = data_info[end]
    
    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    
    all_files = readdir(data_path(ds_save_cy_1))
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*!occursin("shuffleall", candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    println(candidate_filename[which_file])
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    specificity_1 = HDF5.readmmap(file["specificity"])
    valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
    entropy_1 = HDF5.readmmap(file["entropy_all"])
    place_cell_index_1 = HDF5.readmmap(file["place_cell_index"])
    close(file)
    
        
    n_neuron = length(specificity_1)
    
    thresholds = numpy.linspace(0, 1, 100)
    nr_peak = fill(NaN32, n_neuron, length(thresholds))
    @showprogress for i in place_cell_index_1
        which_map = place_map_all_1[:,:,i]
    
        for (i_threshold, threshold) in enumerate(thresholds)
            components_peaks, img_label_valid_gt, valid_components = map_components_peak(which_map; threshold = threshold, components_size_threshold = 20)
            nr_peak[i,i_threshold] = length(components_peaks)
        end
    
    end

    
    h5open(joinpath(data_path(ds_save_cy_1), "unimode_vs_threshold.h5"), "w") do file
        file["nr_peak"] = nr_peak
        file["thresholds"] = thresholds
        file["place_cell_index"] = place_cell_index_1
        
    end;
end

In [ ]:
fig = figure(figsize=(1,1))
for data_info in data_all_exp

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
    experimenter = data_info[end]
    
    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    uni_peak_file = h5open(joinpath(data_path(ds_save_cy_1), "unimode_vs_threshold.h5"))
    nr_peak = read(uni_peak_file,"nr_peak")
    thresholds = read(uni_peak_file,"thresholds")
    place_cell_index_1 = read(uni_peak_file,"place_cell_index")
    close(uni_peak_file)

    uni_peak_ratio = (sum(nr_peak.==1, dims = 1)/length(place_cell_index_1))[1,:];
    
    plot(thresholds .* 100, uni_peak_ratio .* 100)    

end


xlabel("Threshold (%)")
ylabel("Unimode (%)")
xlim(50, 80)
xticks([50, 60, 70, 80])


ylim(0, 100)
fig.savefig(joinpath(fig_path, "unimode_ratio_vs_threshold.pdf"), bbox_inches="tight",transparent = true);